In [ ]:
# Mount do google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall numpy -y
!pip uninstall pandas -y
!pip uninstall scikit-learn -y
!pip uninstall gensim -y

!pip install --upgrade numpy==1.23.5
!pip install --upgrade pandas
!pip install --upgrade scikit-learn
!pip install --upgrade gensim

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 61.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.14.4 requires pandas!=2.1.0,>=1.4, which is not installed.
pymc 5.21.2 requires pandas>=0.24.0, which is not installed.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, which is not installed.
prophet 1.1.6 requires pandas>=1.0.4, which is not installed.
dopamine-rl 4.1.2 requires pandas>=0.24.2, which is not installed.
yfinance 0.2.55 requires pandas>=1.3.0, which is not install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 400.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 90.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires scikit-learn, which is not installed.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, which is not installed.
sklearn-pandas 2.2.0 requires scikit-learn>=0.23.0, which is not installed.
shap 0.47.2 requires scikit-learn, which is not installed.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
bigframes 2.1.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

In [ ]:
# Instanciar os dados
df_train = pd.read_csv('/content/drive/MyDrive/Mackenzie/Projeto Aplicado/df_train_en.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Mackenzie/Projeto Aplicado/df_test_en.csv')

In [ ]:
# visualizar as primeiras 5 linhas dos dados de treino.
df_train.head()

,label,text,language,text_not_stop_words
0,__label__1,the history of torture and execution: i have n...,en,history torture execution: received book order...
1,__label__1,stainless steel shouldn't corrode... should it...,en,stainless steel shouldn't corrode... it?: plea...
2,__label__1,another weak sequel to a once promising franch...,en,weak sequel promising franchise: hellraiser: h...
3,__label__1,"simply awful: contrived, trite, ""bridget jones...",en,"simply awful: contrived, trite, ""bridget jones..."
4,__label__1,"flash: jeff beck's flash in my opinion, was no...",en,"flash: jeff beck's flash opinion, better album..."


In [ ]:
# visualizar as primeiras 5 linhas dos dados de teste.
df_test.head()

,label,text,language,text_not_stop_words
0,__label__1,buyer beware-- too good to be true: i bought t...,en,"buyer beware-- good true: bought tricycle son,..."
1,__label__1,returned to amazon: after reading some great r...,en,returned amazon: reading great reviews i've de...
2,__label__1,poor quality: bungee cord ripped first time i ...,en,poor quality: bungee cord ripped time it. buy ...
3,__label__1,too basic: the book title is misleading. i tho...,en,basic: book title misleading. thought comprehe...
4,__label__1,warranty?: this product lasted 14 months for m...,en,warranty?: product lasted 14 months suitable p...


In [ ]:
# Remoção das colunas redundantes.
df_train.drop(columns=['language', 'text'], inplace=True)
df_test.drop(columns=['language', 'text'], inplace=True)

In [ ]:
df_train.head()

,label,text_not_stop_words
0,__label__1,history torture execution: received book order...
1,__label__1,stainless steel shouldn't corrode... it?: plea...
2,__label__1,weak sequel promising franchise: hellraiser: h...
3,__label__1,"simply awful: contrived, trite, ""bridget jones..."
4,__label__1,"flash: jeff beck's flash opinion, better album..."


In [ ]:
df_test.head()

,label,text_not_stop_words
0,__label__1,"buyer beware-- good true: bought tricycle son,..."
1,__label__1,returned amazon: reading great reviews i've de...
2,__label__1,poor quality: bungee cord ripped time it. buy ...
3,__label__1,basic: book title misleading. thought comprehe...
4,__label__1,warranty?: product lasted 14 months suitable p...


In [ ]:
# Verificar o tipo de dado.
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49871 entries, 0 to 49870
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   label                49871 non-null  object
 1   text_not_stop_words  49871 non-null  object
dtypes: object(2)
memory usage: 779.4+ KB


In [ ]:
# Verificar o tipo de dado.
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199531 entries, 0 to 199530
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   label                199531 non-null  object
 1   text_not_stop_words  199531 non-null  object
dtypes: object(2)
memory usage: 3.0+ MB


# Processing

In [ ]:
# Concatenação dos dados.
all_texts = pd.concat([df_train['text_not_stop_words'], df_test['text_not_stop_words']])

In [ ]:
# Splitando os textos a partir de white space.
tokenized_texts = [text.split() for text in all_texts]

# Word2Vec

In [ ]:
# Instanciando o Word2Vec
model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=100,      # Dimensão do embedding
    window=5,            # Contexto de palavras
    min_count=2,         # Ignorar palavras raras (reduce noise)
    workers=4,
    sg=1,                # Usar Skip-Gram (melhor para dados pequenos)
    hs=0,               # Negative sampling mais eficiente
    negative=5          # Número de amostras negativas
)

In [ ]:
# função para criar os embeddings.
def text_to_embedding(text, model):
    words = text.split()
    # Obter vetores das palavras presentes no vocabulário
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # Vetor zero se nenhuma palavra for encontrada
    return np.mean(word_vectors, axis=0)

In [ ]:
# Encoding das labels.
le = LabelEncoder()
df_train['label'] = le.fit_transform(df_train['label'])
df_test['label'] = le.transform(df_test['label'])

In [ ]:
df_test.head()

,label,text_not_stop_words
0,0,"buyer beware-- good true: bought tricycle son,..."
1,0,returned amazon: reading great reviews i've de...
2,0,poor quality: bungee cord ripped time it. buy ...
3,0,basic: book title misleading. thought comprehe...
4,0,warranty?: product lasted 14 months suitable p...


In [ ]:
df_test['label'].unique()

array([0, 1])

In [ ]:
df_train.head()

,label,text_not_stop_words
0,0,history torture execution: received book order...
1,0,stainless steel shouldn't corrode... it?: plea...
2,0,weak sequel promising franchise: hellraiser: h...
3,0,"simply awful: contrived, trite, ""bridget jones..."
4,0,"flash: jeff beck's flash opinion, better album..."


In [ ]:
df_train['label'].unique()

array([0, 1])

# Modelo

In [ ]:
# Gerar embeddings
X_train = np.array([text_to_embedding(text, model) for text in df_train['text_not_stop_words']])
X_test = np.array([text_to_embedding(text, model) for text in df_test['text_not_stop_words']])
y_train = df_train['label']
y_test = df_test['label']

In [ ]:
# Pipeline com normalização e KNN
pipeline = Pipeline([
    ('scaler', StandardScaler()),      # Normalização integrada
    ('knn', KNeighborsClassifier())    # Modelo
])

In [ ]:
params = {
    'knn__n_neighbors': range(3, 15),         # Evitar k muito baixo
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['cosine', 'euclidean'],   # Cosine para embeddings
    'knn__p': [1, 2]                          # Manhattan ou Euclidiana
}

In [ ]:
# Validação cruzada estratificada
stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=params,
    cv=stratified_cv,
    n_iter=5,                   # Número de combinações testadas
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

In [ ]:
# Treinamento do modelo.
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('knn', KNeighborsClassifier())]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'knn__metric': ['cosine', 'euclidean'],
                                        'knn__n_neighbors': range(3, 15),
                                        'knn__p': [1, 2],
                                        'knn__weights': ['uniform',
                                                         'distance']},
                   random_state=42, scoring='accuracy')

In [ ]:
# Avaliação
print(f"Melhores parâmetros: {random_search.best_params_}")
print(f"Acurácia (TESTE): {accuracy_score(y_test, random_search.predict(X_test)):.4f}")
print(f"Acurácia (TREINO): {accuracy_score(y_train, random_search.predict(X_train)):.4f}")
print("Matriz de Confusão (Teste):")
print(confusion_matrix(y_test, random_search.predict(X_test)))

Melhores parâmetros: {'knn__weights': 'distance', 'knn__p': 1, 'knn__n_neighbors': 11, 'knn__metric': 'cosine'}
Acurácia (TESTE): 0.8152
Acurácia (TREINO): 1.0000
Matriz de Confusão (Teste):
[[19778  5183]
 [ 4033 20877]]
